In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import random 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/Machinehack - Melanoma size prediction/Train.csv')
train.head()

In [ ]:
test = pd.read_csv('/content/gdrive/My Drive/Machinehack - Melanoma size prediction/Test.csv')
test.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.shape

In [ ]:
test.shape

## No missing values so far, lets do EDA

In [ ]:
corr=train.corr()

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(corr, cmap = 'YlGnBu', annot = True, linewidths = 0.5);

In [ ]:
train.corr()['tumor_size'].sort_values()

In [ ]:
train.corr()['tumor_size'][:-1].sort_values().plot(kind='bar');

In [ ]:
# Distribution plots of X_11 and X_15

plt.figure(figsize = (10,8))
sns.distplot(train['tumor_size'], bins = 50,  kde = True);

### The output is skewed towards left, need to transform it to log scale.


In [ ]:
plt.figure(figsize = (10,8))
sns.pairplot(palette = 'BrBG', data = train);

#### Mostly correlated dataset except for the Damage_ratio feature which has a negative correlation.

In [ ]:
plt.figure(figsize = (20,15))
sns.boxplot(data = train);

In [ ]:
tr = train.drop(['exposed_area', 'mass_npea', 'std_dev_malign'], axis =1)

In [ ]:
plt.figure(figsize = (20,15))
sns.boxplot(data = tr);

#### Outlier treatment needs to be done 


## Scaling

In [ ]:
!pip install catboost

## Feature Selection and Model Building 

In [ ]:
# Splitting training dataset into train and test
X = train.copy().drop(['tumor_size'], axis=1).values
y = train['tumor_size']
test = test.copy().values

In [ ]:
print(train.shape)
print(X.shape)
print(y.shape)

# Model Building 

In [ ]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# n_estimators=500, random_state=10, learning_rate=0.9

ada = RandomForestRegressor()
ada_fit = ada.fit(X_train, y_train)

In [ ]:
y_ada_pred_test = ada_fit.predict(X_test)
y_ada_pred_train = ada_fit.predict(X_train)

In [ ]:
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_ada_pred_test))#test RMSE
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_ada_pred_train))#train RMSE

In [ ]:
print('Test RMSE = {}'.format(test_rmse))
print('Train RMSE = {}'.format(train_rmse))

In [ ]:
test_pred = ada_fit.predict(test)

In [ ]:
# save results to csv
subm = pd.DataFrame({'tumor_size': test_pred})
subm = subm[['tumor_size']]

In [ ]:
filename='/content/gdrive/My Drive/Machinehack - Melanoma size prediction/Melanoma_final4.csv'
subm.to_csv(filename, index=False)